```{toggle}
Outline

- intro OS-MM-L01 2-6
    - Overview - Why do we need a memory manager
    - overview of all the chapters
    
**Combine physmem and virt-seg into "Towards paged virtual memory"**
**Reduce the materials for both these sections, get through segmentation faster, and start on paged virtual memory in L01**

- physmem  OS-MM-L01 7-14
    - how program laid out - peter - address space and program loading
    - relocation
    - compaction
    - challenges 

- virt-seg  OS-MM-L01 15-64
    - picture of MMU - sends physical address over bus
    - segmentation hardware - translation
    - what is virtual memory
    - physical mem management
    - swapping
    - compaction 
    - keeping track of free memory
    - challenges
    
**add section at the beginning on interface virtual memory try to get through this in L01**

- virt-paging OS-MM-L02 0-21
    - MMU with some mapping hardware - same picture, but now table
    - page frames and virtual pages
           - keeping track of which memory used
    - mapping & notion of faults
    - page table entry
    - contrast to segmentation, paging of blocks... rather than swapping entire program
    - what we use for:
           - anonymous - program, 
           - file backed - mention file caching
           - page fault
    - concluding remarks
           - once you have multi-level, massive address space. 
           - until now, described HW, and the benefits of having paged virtual memory, next chapters describe some of the key policies, mechanisms 

**L03 is very short, we should start on L04 in L03**
- page table organizations: OS-MM-L02 21-end + L03 0-18
	   - single level.
	   - multi-level page table. 
           - inverted
    - TLB
           - global, local, tagged TLB
           - software managed TLB

**we are covering way too much in L04**

- reclamation OS-MM-L04 1-37
    - describe how we got here ... we run out of memory, ... using 
    - describe all the policies
    - FIFO
    - CLOCK
    - belady anomaly 
    - ... all from peter


- page-sizes OS-MM-L04 38-44
    - why big, why small, supporting multiple


- misc OS-MM-L04 46-47
    - sharing-memory
        - sharing page frames
        - sharing page table pages
    - pinning memory
    - anonymous memory
    - file backed memory
    
- buffer-cache OS-MM-L04 52- 62
    - most memory used. 
    
- MM dynamics  OS-MM-L04 63- 68 
    - anonymous - zero filled first access.. 
    - file backed 
    - COW

- realworld OS-MM-L05 
    - Linux implementation

- concl
    - Review of MM
```

(cont:mm:intro)=
# Introduction
The portion of the OS kernel that manages and tracks memory is the memory management system.  Today, paged virtual memory provides each process with a virtual memory abstraction of massive contiguous memory that starts at address `0x0`, while controlling what physical memory is used by the operating system and different user processes at the granularity of fixed chunks (e.g., 4KB) of memory (```pages```). 

We [first](cont:mm:beforepaging) motivate today's paged virtual memory by describing how early operating systems worked that supported only physical and segmented virtual memory.  We [then](cont:mm::virt-paging) describe how paged virtual memory works today. We then discuss one of the major uses of memory, the [buffer cache](cont:mm:buffer-cache), that uses system memory to cache file contents to avoid file requests having to go to slow storage.   Finally, we provide some details on how memory management works in [Linux today](cont:mm:realworld). 



To paraphrase [Parkinson's law](https://en.wikipedia.org/wiki/Parkinson%27s_law) programs expand to fill the memory available to hold them, and home computers have millions of times more memory than they had when early computers were very expensive and very large.  

A virtual address space includes 2 basic types of regions: anonymous and file-backed.  An anonymous virtual memory region is one that is not backed up by any file.  These include the stacks, heaps and other dynamically and statically allocated regions of a program.  A file-backed virtual memory region is one that is backed up by a file.  These include the text and initialized data sections of a program.
Much more on this later.

Today, most computers support paged virtual memory, but before we get into that, we will look at the motives that lead to today's memory management by first discussing systems without any virtual memory and and then discussing with segmented virtual memory.  


## Physical Memory/No Virtual Memory
On a system without virtual memory there is one physical address space and programs are loaded into physical memory, allocate physical memory as needed and directly access those physical memory locations or addresses.  If more than one program is running at the same time they are simply loaded into a different physical address and allocate different physical addresses.  This means every program must either know the physical address that it is running or it must be relocated when it is loaded. 

## Virtual Memory
What is most desirable and usable is a virtual memory model where every program would run in a process that has a unique but identical virtual address space.  A virtual address space would ideally start at address zero and extend up to some very large address.  Since every process appears to have the same virtual address space programs can be loaded at the same virtual address and therefore do not need to to be relocated.  There are 2 basic virtual memory models: Segmented Virtual Memory and Paged Virtual Memory.

### Segmented Virtual Memory
A simple segmented virtual address space starts at location zero and extend up to some upper limit. Its implemented in hardware using base and offset registers.  Every memory reference is virtual and is translated to a physical address in hardware by adding the base register to the virtual address and qualified by insuring the result is less than the offset register.

### Paged Virtual Memory
A paged virtual memory model splits both the virtual address space and physical memory into fixed size pieces knows as virtual pages and physical page frames respectively.  The mapping of virtual pages to physical page frames is accomplished via an array of page frame numbers known as a page table.  There is one page table entry for each virtual page and only then pages that are being used contain valid page table entries and therefore map physical page frames.

#### The virtual address space
The virtual address space of every process is typically very large and fixed in size, for example the virtual address space for every process running on an x86_64 system is 2^48 or 258TB.  By default the entire virtual address space is not accessible to programs running in it.  The only portions or regions that are accessible are those that the program requested via system calls(mmap and sbrk) and the kernel granted access to.  So the virtual address space is sparsely populated with valid regions and those are either anonymous regions or file-backed regions.  Anonymous regions are any region that is not backed by a file, for example; stacks, heaps, BSS, malloc()'d and sbrk()'d regions.   File-backed regions are sections of a file that are mapped into the virtual address space.  Once the valid regions have been created physical memory pages are mapped into the virtual pages on demand therefore those regions are sparsely populated with physical memory.

Paged virtual memory aka paging introduces the possibility of several additional highly desirable features or pros to the memory management system but also introduces several additional challenges or cons.

#### Pros
 - Several huge virtual address spaces can be much larger than physical memory via over-committing physical memory.
 - Total isolation of each virtual address space from all others.
 - Sharing of selective regions of memory between different virtual address spaces is easy.
 - Use of memory pages to cache file system data and meta-data is possible.
 - Very large and sparse virtual address space requires page fault logic that maps physical memory on-demand.
 
#### Cons
 - Over-committing physical memory requires sophisticated page reclaim logic.
 - Supporting multiple page sizes on different architectures complicates an already complex memory management system.
 - Very large virtual address space requires large and complex multi-level page table design.


The next chapter describes...

- [virt-seg](cont:mm:virt-seg)
- [virt-paging](cont:mm:virt-paging)
- [page-tables](cont:mm:page-tables)
- [reclamation](cont:mm:reclamation)
- [page-size](cont:mm:page-size)
- [misc](cont:mm:misc)
- [buffer-cache](cont:mm:buffer-cache)
- [mem-dynamics](cont:mm:mmdyn)
- [realworld](cont:mm:realworld)
- [concl](cont:mm:concl)